In [20]:
from Models.BlackBox.lmart.lgbmranker_class import LGBMRanker_class
import pandas as pd
%reload_ext autoreload
%autoreload 2

In [11]:
lMart_parameter = dict(
    boosting_type=["dart", "gbdt"],
    num_leaves=[2, 5, 10, 20, 30, 40],
    max_depth=[-1],
    n_estimators=[75, 80, 100, 150, 200],
    learning_rate=[0.02, 0.05, 0.08, 0.1, 0.15, 0.2],
    reg_lambda=[0.00005, 0.0001, 0.0002, 0.0003]
)
gridsearch_parameters = dict(
    path="./../../../outputs/scores/",
    name="0",
    nDCG_at=15
)

# Grid-search

In [21]:
gs = LGBMRanker_class(**gridsearch_parameters)

In [13]:
best_ = gs.grid_search(lMart_parameter)
gs.save_model(best_[0],"LGBMRanker")

100%|██████████| 1440/1440 [08:05<00:00,  2.97it/s, nDCG_15=0.998]


# Metric evaluation

In [22]:
best_model = gs.load_model("./saved_models/LGBMRanker")

In [23]:
# nDCG
nDCG_train = gs.eval_model(model=best_model, dt=gs.train, qIds=gs.qIds_train, nDCG_at=[1,10,15])
nDCG_valid = gs.eval_model(model=best_model, dt=gs.valid, qIds=gs.qIds_valid, nDCG_at=[1,10,15])
nDCG_test = gs.eval_model(model=best_model, dt=gs.test, qIds=gs.qIds_test, nDCG_at=[1,10,15])

other_evals_tr = gs.other_eval(df=gs.test, qIds=gs.qIds_test, nDCG_at=[1,10,15])

display(pd.DataFrame([nDCG_train,nDCG_valid,nDCG_test],index=["Training","Validation","Test"]))

print("Test set")
display(pd.DataFrame(other_evals_tr).T)

,nDCG@1,nDCG@10,nDCG@15
Training,1.0000,0.9981,0.9987
Validation,0.9950,0.9968,0.9978
Test,0.9888,0.9946,0.9964


Test set


,nDCG@1,nDCG@10,nDCG@15
random_permutation,0.7204,0.7581,0.7872
perfect_nDCG,1.0000,1.0000,1.0000
worste_nDCG,0.3404,0.4935,0.5814
